# Actividad: Clasificación con máquina de vectores de soporte y redes de neuronas (Mobile Price Classification)

## Introducción

El problema orginal tiene una breve descripción contextual (.sic):

> Bob has started his own mobile company. He wants to give tough fight to big companies like Apple,Samsung etc.
>
> He does not know how to estimate price of mobiles his company creates. In this competitive mobile phone market you cannot simply assume things. To solve this problem he collects sales data of mobile phones of various companies.
> 
> Bob wants to find out some relation between features of a mobile phone(eg:- RAM,Internal Memory etc) and its selling price. But he is not so good at Machine Learning. So he needs your help to solve this problem.
>
> In this problem you do not have to predict actual price but a price range indicating how high the price is. \citep{Sharma2018}

La variable objetivo es la variable "price_range". En este análisis no se usarán los dos datasets, solo [train.csv](https://www.kaggle.com/iabhishekofficial/mobile-price-classification#train.csv) que corresponde a los datos de entrenamiento.

### Bibliotecas a utilizar

En el presente análisis se comparará el funcionamiento de una SVM y una red neuronal básica por lo tanto se requieren las siguientes bibliotecas.

In [ ]:
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Carga de dataset y análisis descriptivo de datos

In [ ]:
df_train = pd.read_csv("ds/train.csv")

Según los datos proporcionados no existen valores perdidos. Además todas las columnas tienen valores numéricos.

In [ ]:
df_train.info()

In [ ]:
df_train.describe().transpose() 

Aún así parece que no todas las variables son realmente numéricas, pues no caen en valores continuos. Como ejemplo es posible observar la columna objetivo, la cual por medio de números distingue cuatro categorías. Debido a la falta de metadatos en el dataset no es posible saber con certeza a que corresponde cada una, sin embargo, una clasificación tradicional es: el segmento de entra (0), la gama media (1), la gama alta (2) y la gama premium (3).

In [ ]:
sns.countplot(x='price_range', data=df_train)
plt.title('Frecuencias de price_range', fontsize=12)

### Variables categóricas

Es importante observar entonces que datos son categóricos y examinarlos como tal.

In [ ]:
for i in df_train.columns:
    print(f'Valores posibles de {i.title()}: {df_train[i].unique()}')

Las variables categóricas parecen ser: `blue`, `dual_sim`, `four_g`, `three_g`, `touch_screen`, `wifi` y `price_range`. Debido a que `price_range` es nuestra variable objetivo, en seguida se grafican las frecuencias de las otras variables categóricas tomando en cuenta el rango de precios.

In [ ]:
cv=['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
fig=plt.figure(figsize=(24,12))
plt.title('Frecuencias de variables categóricas tomando en cuenta el rango de precios',fontdict={'fontsize':20})
plt.axis('off')

for i in range(len(cv)):
    fig.add_subplot(2,3,i+1)
    sns.countplot(data=df_train,x=cv[i], hue='price_range')
    plt.legend(bbox_to_anchor=(1.02, 1), borderaxespad=0)

Es posible observar que los teléfonos más costosos cuentan con más de estas características, salvo en el caso de `touch_screen`, donde no hay diferencia frecuencial visible. La variable `three_g` parace ser más significativa ya que la división es un poco más clara. También hay que matizar que esta diferencia no es suficiente para distinguir invariablemente las categorías de rango de precio.

### Variables numéricas

Se repetirá el procedimiento anterior procedimiento para las variables que son efectivamente numéricas pues servirá para observar el contraste entre los tipos de variables.

In [ ]:
nv=['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time']
fig=plt.figure(figsize=(24,30))
plt.title('Frecuencia de variables numéricas tomando en cuenta el rango de precios')
plt.axis('off')

for i in range(len(nv)):
    fig.add_subplot(7,2,i+1)
    sns.kdeplot(data=df_train, x=nv[i], hue='price_range')

La dentro de las gráficas anteriores lo más destacado es aquella que representa la variable `ram`. En dicha representación se distinguen muy claramente los segmentos de precio, es posible pensar que que la variable `ram`, tenga una correlación alta respecto a `price_range`, mientras otras como `clock_speed` sean de muy poca relevancia.

## Matriz de correlación

La matriz es indispensable para distinguir la importancia de las variables y así pensar en un mejor modelo.

In [ ]:
plt.figure(figsize=(20,8),dpi=80)
corrmat = df_train.corr()
sns.heatmap(corrmat, cmap='coolwarm', vmax=.8, fmt='.1f', annot=True)

In [ ]:
df_train.corr()['price_range'].sort_values(ascending=False)[1:21]

Tal y como se dijo anteriormente, la variable `ram` tiene una alta correlación con la variable objetivo.

## SVM

Se implementará una SVM. Al trabajar con métodos supervisados es necesario dividir el dataframe en dos partes.

In [ ]:
train, test = train_test_split(df_train, test_size=0.2)
predictors = cv + nv
#predictors = ['ram', 'battery_power', 'px_width', 'px_height', 'int_memory']
target = ['price_range']

Ahora, ante la diversidad de hiperparámetros se crea la SVM al mismo tiempo que se búsca el mejor modelo posible. Este proceso es muy lento, a continuación se muestran solo los hiperparámetros más relevantes, otros fueron descartados pues en una ejecución individual dieron resultados deficientes.

In [ ]:
parameters = [{'kernel': ['rbf', 'linear'], 'gamma': ['scale', 'auto'], 'C': [0.5, 1.0, 1.5, 2.0, 2.5]}]
svmc = RandomizedSearchCV(svm.SVC(decision_function_shape='ovr'), param_distributions=parameters, cv=5, scoring='accuracy')
svmc.fit(train[predictors], np.ravel(train[target]))
svmc.best_params_

In [ ]:
svm_predicted = svmc.predict(test[predictors])
crosstab = sns.heatmap(pd.crosstab(test[target].values.ravel(), svm_predicted),
                cmap='gist_yarg', vmax=.8, fmt='g', annot=True)
crosstab.set_xlabel('Predicciones')
crosstab.set_ylabel('Reales')
print("Accuracy: ", metrics.accuracy_score(svm_predicted, np.ravel(test[target])))

Los resultados de la SVM resultan bastante prometedores, la presición alcanzada es muy alta, en alguna ejecución llegó a ser del 99%, sin embargo, se consigue en pocos casos.

## Red Neuronal

La red neuronal ha resultado ser más complicada de implementar debido a que requiere el diseño de la las capaz de la red. Aquí se presenta una estructura básica. \citep{rickytb2021, Keras2022}

In [ ]:
nn = Sequential()
nn.add(Dense(units=16, input_dim=20, activation='relu'))
nn.add(Dense(units=12, activation='relu'))
nn.add(Dense(units=4, activation='softmax'))

In [ ]:
nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
nn.fit(train[predictors], np.ravel(train[target]), epochs=550, batch_size=4, verbose=0)

In [ ]:
nn_pred = nn.predict(test[predictors])

El resultado de la red neuronal no es legible inmediatamente, se requiere convertir las prediccinose a sus respectivas etiquetas. \citep{rickytb2021}

In [ ]:
# Convertir las predicciones a sus respectivas etiquetas
def pred_to_label(predictions):
    pred = list()
    for i in range(len(predictions)):
        pred.append(np.argmax(predictions[i]))
    return pred

In [ ]:
nn_predicted = pred_to_label(nn_pred)
crosstab = sns.heatmap(pd.crosstab(test[target].values.ravel(), np.array(nn_predicted)),
                cmap='gist_yarg', vmax=.8, fmt='g', annot=True)
crosstab.set_xlabel('Predicciones')
crosstab.set_ylabel('Reales')
print("Accuracy: ", metrics.accuracy_score(nn_predicted, np.ravel(test[target])))

La red neuronal, no ha alcanzado la gran presición que logró la SVM. Si se considera que este modelo fue creado a partir de la documentación básica de la biblioteca que lo implementa y no ha sido totalmente personalizado, es posible pensar que el resultado es positivo y podría alcanzar mejores predicciones con mayor tiempo.

## Comparación entre métodos

Como ya se ha visto, la precisión conseguida por la SVM es superior a la obtenida en la red neuronal. Habrá que considerar que la potencia de computo ha limitado la red neuronal, por lo tanto, para comparar mejor los algoritmos se usarán las métricas proporcionadas por `sklearn`.

In [ ]:
print(metrics.classification_report(test[target].values.ravel(), svm_predicted))

In [ ]:
print(metrics.classification_report(test[target].values.ravel(), nn_predicted))

## Conclusión

En el presente análisis se utilizó un dataset llamado _Mobile price classification_. Se realizó un estudio estadístico general del dataset y se encontraron dos mil muestras en el mismo.

Se decidió aplicar dos algoritmos avanzados: SVM y redes neuronales. Estos algoritmos requieren trabajo al afinar los hiperparámetros de cada modelo, si bien son herramientas muy poderosas, también hay que decir que ajustarlas para solucionar el problema puede ser más costoso que usar otro algoritmo más básico.

En ejemplos sencillos como el resuelto en este documento, dichos hiperparámetros son ajustables con cierta sencillez. Aún así, la fase de entrenamiento es más lenta si se considera que el dataset es de solo 2000 muestras. Si a lo anterior se sumaran muestras con overlapping, la SVM particularmente tenderá a cometer un mayor número de errores. La red neuronal incluso sugería el uso de una tarjeta gráfica.

Si el dataset tiene muchas dimensiones, la SVM puede ser una excelente alternativa. Aunque no podrán ser visualizadas gráficamente, este algoritmo lidiará bien con el dataset incluso si el número de dimensiones supera el número de muestras, como en el procesamiento de imágenes. Por su parte, las redes neuronales pueden ser más útiles con información más desestructurada o en datos más complejos sin querer abusar del método prueba/error.